<a href="https://colab.research.google.com/github/ashmibanerjee/email-automation/blob/main/Email_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 0: Imports and Utilities

In [1]:
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import configparser
from datetime import datetime

import re
import os

In [2]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive/"
drive.mount(ROOT)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd drive/My\ Drive/personal/Events\ \&\ Presentations/PyconPortugal22/email-automation/

/content/drive/My Drive/personal/Events & Presentations/PyconPortugal22/email-automation


In [4]:
regex = re.compile(r"([-!#-'*+/-9=?A-Z^-~]+(\.[-!#-'*+/-9=?A-Z^-~]+)*|\"([]!#-[^-~ \t]|(\\[\t -~]))+\")@([-!#-'*+/-9=?A-Z^-~]+(\.[-!#-'*+/-9=?A-Z^-~]+)*|\[[\t -Z^-~]*])")

def isValid(email):
    if re.fullmatch(regex, email):
        print("Valid email")
        return True
    print("Invalid email")
    return False

### Step 1: Getting Sender Info from Config File

In [9]:
config = configparser.ConfigParser()
config.read("settings.ini")

['settings.ini']

In [10]:
sender_email = config["SenderAccountInfo"]["EMAIL"]
sender_name = config["SenderAccountInfo"]["NAME"]
cc = config["SenderAccountInfo"]["CC"]
app_password = config["SenderAccountInfo"]["APP_PASSWORD"]

### Step 2: Get email Subject

In [11]:
def get_mail_subject():
    return "Testing Automated Emails"

### Step 3: Get email Body

In [12]:
def get_mail_body(receiver_first_name: str, sender_name: str):
    msg_body = f"Dear {receiver_first_name}, \nThis email was automatically sent using Python. \n" \
               f"Hope you like it. \n Cheers, \n {sender_name}  "
    return msg_body

### Step 4: Get email Attachment

In [22]:
def get_email_attachment(file_name: str = "test_image.png"):
    file_path = os.getcwd() + "/assets/"
    attachment_file = file_path + file_name
    return attachment_file

### Step 5: Get Recipient Info

In [14]:
receiver_name = input("Enter recipient full name (First name Last Name): ")

try:
    receiver_first_name, receiver_last_name = receiver_name.split(" ")
except ValueError:
    receiver_first_name, receiver_last_name = receiver_name, ""

while True:
    receiver_email = input("Enter recipient Email: ")
    if isValid(receiver_email):
        break

print(f"\n Recipient Infos: \n\tFirst Name: {receiver_first_name} \n\tLast Name: {receiver_last_name} \n\tEmail: {receiver_email}")

Enter recipient full name (First name Last Name): dghjh
Enter recipient Email: ashmi.banerjee@tum.de
Valid email

 Recipient Infos: 
	First Name: dghjh 
	Last Name:  
	Email: ashmi.banerjee@tum.de


### Step 6: Prepare Email

In [15]:
email = MIMEMultipart()
email["From"] = sender_email
email["To"] = receiver_email
email["Subject"] = get_mail_subject()
# email["Cc"] = cc
email_body = get_mail_body(receiver_first_name, sender_name)

In [16]:
# Add body and attachment to email
email.attach(MIMEText(email_body, "html"))

In [23]:
# Add email attachment
attachment_file_name = "test_image.png"
attachment = open(get_email_attachment(), "rb")

/content/drive/MyDrive/personal/Events & Presentations/PyconPortugal22/email-automation


In [24]:
# instance of MIMEBase and named as p
p = MIMEBase('application', 'octet-stream')

 # To change the payload into encoded form
p.set_payload(attachment.read())

# encode into base64
encoders.encode_base64(p)

p.add_header('Content-Disposition', "attachment; filename= %s" % attachment_file_name)

# attach the instance 'p' to instance 'msg'
email.attach(p)

email = email.as_string()

### Step 7: Send Email 

In [26]:
# Create SMTP session for sending the mail
# use gmail with port
session = smtplib.SMTP('smtp.gmail.com', 587)

# enable security
session.starttls()

# login with mail_id and password
session.login(sender_email, app_password)

# send email
session.sendmail(sender_email, receiver_email, email)

# quit session
session.quit()

Mail Sent to dghjh at ashmi.banerjee@tum.de at 2022-09-02 09:25:53.271420


### Step 8: Celebrate !! 🎊

In [27]:
print(f"Mail Sent to {receiver_name} at {receiver_email} at {datetime.now()}")

Mail Sent to dghjh at ashmi.banerjee@tum.de at 2022-09-02 09:31:09.430216
